In [2]:
########################start##############################
import pandas as pd
df=pd.read_csv('./datas/Data_new.csv',index_col=0)
df.index=pd.to_datetime(df.index,format='%Y-%m-%d') 
df_train=df.loc['2020-01-07':'2021-04-02'] 
df_test=df.loc['2021-04-02':'2021-04-23']

name=['cityin','cityto','incity','city']
name1=['cityin','cityto','incity']
x_train=df_train.drop(name,axis=1).values  
y_train=df_train[name1].values
x_test=df_test.drop(name,axis=1).values  
y_test=df_test[name1].values
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

#####################标准化##############################
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_train = ss.transform(x_train) 
x_test = ss.transform(x_test)

(57504, 43) (19133, 43) (57504, 3) (19133, 3)


In [3]:
import csv
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=1.5)  # 解决Seaborn中文显示问题并调整字体大小
sns.set_style("white")
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:                   
        os.makedirs(path)            
        print("---  new folder...  ---")
        print("---  OK  ---")
    else:
        print("---  There is this folder!  ---")
mkdir("./results/")        
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,mean_squared_log_error

def calculate(y_true, y_predict, n, p):
    y_true = y_true
    y_predict = y_predict
    mse = mean_squared_error(y_true, y_predict)
    rmse = np.sqrt(mean_squared_error(y_true, y_predict))
    mae = mean_absolute_error(y_true, y_predict)
    r2 = r2_score(y_true, y_predict)
    mad = median_absolute_error(y_true, y_predict)
    mape = np.mean(np.abs((y_true - y_predict) / y_true)) * 100
    r2_adjusted = 1-((1-r2)*(n-1))/(n-p-1)
    # print('MSE: ', mse)
    # print('RMSE: ', rmse)
    # print('MAE: ', mae)
    # print('R2: ', r2)
    # print('MAD:', mad)
    # print('MAPE:', mape)
    # print('R2_Adjusted: ',r2_adjusted)
    return mse,rmse,mae,r2,mad,mape,r2_adjusted
# 将参数和评估结果写入文件
def write_csv_result(path_1,path_2,all_metrics,all_parameter):
    with open(path_1,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_metrics)
    with open(path_2,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_parameter)


---  There is this folder!  ---


In [ ]:
from sklearn.ensemble import BaggingRegressor
import csv
import os
import numpy as np
from sklearn.multioutput import RegressorChain


def Bagging(train_x,test_x,train_y,test_y,name):
    path_a = "./results/"  + "Bagging_" + name + "_" + "assess.csv"
    path_p = "./results/"  + "Bagging_" + name + "_" + "parameter.csv"
    path_b = "./results/"  + "Bagging_" + name + "_" + "best.csv"
    model_path = "./results/"  + "Bagging_" + name + "_" + "best.m"
    # 人工设置网格搜索的范围
    n_estimators = [10+(5*i) for i in range(200)]
    max_samples = [0.7,0.8,0.9,1.0]
    max_features = [0.7,0.8,0.9,1.0]
    all_len = len(n_estimators) * len(max_samples) * len(max_features)
    num=1
    # 用于重启训练模型，提高效率，不重复跑相同的实验
    if(os.path.exists(path_a)):
        data = pd.read_csv(path_a)
        if(data.shape[0]>1):
            nums = int(data.values[-1,0])
        else:
            nums = 0
    else:
        nums = 0
        col_a = ['num','mse','rmse','mae','r2','mad','mape','r2_adjusted']
        col_p = ['num','n_estimators','max_samples','max_features']
        write_csv_result(path_a,path_p,col_a,col_p)
    # 用于保存最好的模型
    if(os.path.exists(path_b)):
        best_result = 10*10**30
    else:
        with open(path_b,"a",encoding="utf-8",newline="")as f:
            f = csv.writer(f)
            f.writerow(['num','mse','rmse','mae','r2','mad','mape','r2_adjusted'])
        best_result = 10*10**30
    # 网格搜索
    for mf in max_features:
        for ma in max_samples:
            model = BaggingRegressor(warm_start=True,n_estimators=1, random_state=17,max_samples=ma,max_features=mf)
            for n in n_estimators:
                if(nums>=num):
                    num = num+1
                else:
                    print("Bagging train...{}/{}".format(num,all_len))
                    model.n_estimators = n
                    wrapper = RegressorChain(model)
                    wrapper.fit(train_x,train_y)
                    pred_test = wrapper.predict(test_x)
#     
                    y_test=test_y       
                    y_test1=[]
                    for i in y_test:
                        y_test1.append(i[0])
                    pred_test1=[]
                    for i in pred_test:
                        pred_test1.append(i[0])
                    y_test2=[]
                    for i in y_test:
                        y_test2.append(i[1])
                    pred_test2=[]
                    for i in pred_test:
                        pred_test2.append(i[1])
                    y_test3=[]
                    for i in y_test:
                        y_test3.append(i[2])
                    pred_test3=[]
                    for i in pred_test:
                        pred_test3.append(i[2])
                    pred_test1 = np.array(pred_test1).reshape(-1,1)
                    y_test1 = np.array(y_test1).reshape(-1,1)
                    sample_n = pred_test1.shape[0]
                    feature_n = x_test.shape[1]
                    mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1 = calculate(y_test1,pred_test1,sample_n,feature_n)
                

                    pred_test2 = np.array(pred_test2).reshape(-1,1)
                    y_test2 = np.array(y_test2).reshape(-1,1)
                    sample_n = pred_test2.shape[0]
                    feature_n = x_test.shape[1]
                    mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2 = calculate(y_test2,pred_test2,sample_n,feature_n)
                    

                    pred_test3 = np.array(pred_test3).reshape(-1,1)
                    y_test3 = np.array(y_test3).reshape(-1,1)
                    sample_n = pred_test3.shape[0]
                    feature_n = x_test.shape[1]
                    mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3 = calculate(y_test3,pred_test3,sample_n,feature_n)
    
                    mse,rmse,mae,r2,mad,mape,r2_adjusted = (mse1+mse2+mse3)/3,(rmse1+rmse2+rmse3)/3,(mae1+mae2+mae3)/3,(r21+r22+r23)/3,(mad1+mad2+mad3)/3,(mape1+mape2+mape3)/3,(r2_adjusted1+r2_adjusted2+r2_adjusted3)/3
                    all_m = [num,mse,rmse,mae,r2,mad,mape,r2_adjusted]
                    all_p = [num,ma,mf,n]
                    # print(all_m)
                    write_csv_result(path_a,path_p,all_m,all_p)              
                    if(rmse < best_result):
                        joblib.dump(filename=model_path,value=model,compress=True)
                        with open(path_b,"w",encoding="utf-8",newline="")as f:
                            f = csv.writer(f)
                            f.writerow([num,mse,rmse,mae,r2,mad,mape,r2_adjusted])
                            f.writerow([num,mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1])
                            f.writerow([num,mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2])
                            f.writerow([num,mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3])
                        best_result = rmse
                        print("current best result, mse:{},mae:{},rmse:{},mad:{},r2:{},mape:{},r2 adjusted:{}".format(mse,mad,rmse,mad,r2,mape,r2_adjusted))
                    print("end....",num)
                    num = num+1
                    print("--------------------------------") 


name='Migration'
Bagging(x_train,x_test,y_train,y_test,name)

Bagging train...85/3200
current best result, mse:0.1820622329968823,mae:0.09619416339017749,rmse:0.4245777405017274,mad:0.09619416339017749,r2:0.8278024254990796,mape:13.27407017461075,r2 adjusted:0.8274145321728948
end.... 85
--------------------------------
Bagging train...86/3200
current best result, mse:0.18191654445534522,mae:0.09608545074329312,rmse:0.4244355826300596,mad:0.09608545074329312,r2:0.8279100216821437,mape:13.27377982333382,r2 adjusted:0.8275223707277894
end.... 86
--------------------------------
Bagging train...87/3200
end.... 87
--------------------------------
Bagging train...88/3200
end.... 88
--------------------------------
Bagging train...89/3200
end.... 89
--------------------------------
Bagging train...90/3200
end.... 90
--------------------------------
Bagging train...91/3200
end.... 91
--------------------------------
Bagging train...92/3200
end.... 92
--------------------------------
Bagging train...93/3200
end.... 93
--------------------------------
Bag

current best result, mse:0.1807840505437124,mae:0.09532105798845607,rmse:0.42313842480094116,mad:0.09532105798845607,r2:0.8290871396849585,mape:13.283273336546076,r2 adjusted:0.8287021403139306
end.... 153
--------------------------------
Bagging train...154/3200
end.... 154
--------------------------------
Bagging train...155/3200
end.... 155
--------------------------------
Bagging train...156/3200
current best result, mse:0.18077036272681227,mae:0.09546798487473618,rmse:0.4231232418220973,mad:0.09546798487473618,r2:0.8290997652076496,mape:13.286207646031995,r2 adjusted:0.8287147942769528
end.... 156
--------------------------------
Bagging train...157/3200
current best result, mse:0.18072045812297555,mae:0.09549930569057868,rmse:0.4230632836755867,mad:0.09549930569057868,r2:0.829156057683485,mape:13.286084800986814,r2 adjusted:0.828771213557569
end.... 157
--------------------------------
Bagging train...158/3200
current best result, mse:0.18071720637156052,mae:0.09556123203074463,r

end.... 248
--------------------------------
Bagging train...249/3200
end.... 249
--------------------------------
Bagging train...250/3200
end.... 250
--------------------------------
Bagging train...251/3200
end.... 251
--------------------------------
Bagging train...252/3200
end.... 252
--------------------------------
Bagging train...253/3200
end.... 253
--------------------------------
Bagging train...254/3200
end.... 254
--------------------------------
Bagging train...255/3200
end.... 255
--------------------------------
Bagging train...256/3200
end.... 256
--------------------------------
Bagging train...257/3200
end.... 257
--------------------------------
Bagging train...258/3200
end.... 258
--------------------------------
Bagging train...259/3200
end.... 259
--------------------------------
Bagging train...260/3200
end.... 260
--------------------------------
Bagging train...261/3200
end.... 261
--------------------------------
Bagging train...262/3200
end.... 262
--------

end.... 366
--------------------------------
Bagging train...367/3200
